# მანქანური სწავლება - ფინალური დავალება

კომპანია, რომელშიც მუშაობთ როგორც მონაცემთა მეცნიერი, თანამშრომლობს ერთ-ერთ საავადმყოფოსთან, სადაც იკვლევენ დიაბეტს. ამ საავადმყოფომ შეაგროვა
ქალი პაციენტების მონაცემები. მათი მთავარი მიზანია დიაბეტის ადრეული დიაგნოსტირება პაციენტებში. თქვენი ამოცანაა დავალებასთან თანდართული მონაცემთა ფაილის ანალიზი, მოდელის შექმნა და დატრენინგებული მოდელის შეფასება.

## კოდების გაშვებისთვის საჭირო ბიბლიოთეკების იმპორტები

In [ ]:
import datetime
from itertools import product

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import umap
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

## სტილის არჩევა გრაფიკებისთვის

In [ ]:
sns.set_style("white")
sns.set_palette("Paired")

## მონაცემთა ანალიზი

ფაილი შედგება 9 სვეტისგან:
* **Pregnancies** - რამდენჯერ იყო პაციენტი ორსულად
* **Glucose** - პლაზმური გლუკოზის კონცენტრაცია
* **BloodPressure** - დიასტოლური არტერიული წნევა
* **SkinThickness** - ტრიცეფსის კანის ნაკეცის სისქე
* **Insulin** - ინსულინის დონე
* **BMI** - სხეულის მასის ინდექსი
* **DiabetesPedigreeFunction** - დიაბეტის ალბათობა ოჯახის ისტორიაზე დაყრდნობით
* **Age** - პაციენტის ასაკი
* **Outcome** - ტესტის შედეგები (1 - პაციენტს აქვს დიაბეტი, 0 - არ აქვს)

In [ ]:
df = pd.read_csv("./data/diabetes.csv")
df.head()

ვნახოთ მონაცემების ზომა:

In [ ]:
df.shape

ვნახოთ მონაცემთა ტიპები:

In [ ]:
df.dtypes

ვნახოთ მარტივი აღწერითი სტატისტიკა:

In [ ]:
df.describe()

აუცილებელია დავაკვირდეთ გამოტოვებულ მონაცემებს:

In [ ]:
df.info()

In [ ]:
pd.DataFrame(
    {
        "Number of missing data": df.isna().sum(),
        "Percentage of missing data": (df.isna().sum() / len(df) * 100)
        .round(2)
        .astype(str)
        + "%",
    },
    index=df.columns,
)

რადგანაც ინსულინის დონის მონაცემების თითქმის 50% გამოტოვებულია, დავაკვირდეთ რომელი პაციენტებისთვისაა გამოტოვებული - დიაბეტის მქონე თუ ჯანმრთელი პაციენტებისთვის, იქნებ ჯანმრთელებისთვის არის გამოტოვებული მხოლოდ:

In [ ]:
df[df["Insulin"].isna()]["Outcome"].value_counts()

როგორც ვხედავთ, ორივე კატეგორიის პაციენტებისთვისაა გამოტოვებული. რადგანაც ინსულინის დონე ლოგიკურად კავშირშია დიაბეტთან, ამიტომ ამ ცვლადის მახასიათებლად შენარჩუნებას ყველანაირად ვეცდებით - ანუ ამოვავსებთ გამოტოვებულ მნიშვნელობებს მასში.

სამიზნე ცვლადი არის **Outcome**, შესაბამისად, დავაკვირდეთ რამდენად დაბალანსირებულია:

In [ ]:
df["Outcome"].value_counts()

In [ ]:
(df["Outcome"].value_counts(normalize=True) * 100).round(2).astype(str) + "%"

ვნახოთ თითოეული მახასიათებლის განაწილება:

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(14, 16))

sns.histplot(df, x="Pregnancies", hue="Outcome", kde=True, ax=axes[0, 0])
sns.histplot(df, x="Glucose", hue="Outcome", kde=True, ax=axes[0, 1])
sns.histplot(df, x="BloodPressure", hue="Outcome", kde=True, ax=axes[1, 0])
sns.histplot(df, x="SkinThickness", hue="Outcome", kde=True, ax=axes[1, 1])
sns.histplot(df, x="Insulin", hue="Outcome", kde=True, ax=axes[2, 0])
sns.histplot(df, x="BMI", hue="Outcome", kde=True, ax=axes[2, 1])
sns.histplot(df, x="DiabetesPedigreeFunction", hue="Outcome", kde=True, ax=axes[3, 0])
sns.histplot(df, x="Age", hue="Outcome", kde=True, ax=axes[3, 1])

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(14, 16))

sns.boxplot(df, x="Pregnancies", ax=axes[0, 0])
sns.boxplot(df, x="Glucose", ax=axes[0, 1])
sns.boxplot(df, x="BloodPressure", ax=axes[1, 0])
sns.boxplot(df, x="SkinThickness", ax=axes[1, 1])
sns.boxplot(df, x="Insulin", ax=axes[2, 0])
sns.boxplot(df, x="BMI", ax=axes[2, 1])
sns.boxplot(df, x="DiabetesPedigreeFunction", ax=axes[3, 0])
sns.boxplot(df, x="Age", ax=axes[3, 1])

plt.tight_layout()
plt.show()

დავაკვირდეთ მახასიათებლების კორელაციას:

In [ ]:
plt.figure(figsize=(8, 8))
sns.heatmap(df.corr(), annot=True, square=True, cmap="Blues")
plt.show()

გამოვსახოთ მონაცემები ორ განზომილებიან გრაფიკზე განზომილების შემცირების ტექნიკის საშუალებით, თუმცა ერთი პრობლემაა - გამოტოვებული მნიშვნელობები. რადგანაც მხოლოდ და მხოლოდ ვიზუალურად გვინდა დაკვირვება, მაშინ ასეთი რამ ვქნათ, დროებით შევავსოთ გამოტოვებული ადგილები მთლიანი მონაცემების სტატისტიკაზე დაყრდნობით. როგორც გრაფიკებიდან ვხედავთ, აუთლაიერები გვაქვს უმეტესობა მახასიათებელში, თუმცა ზოგიერთი მათგანის აღწერით სტატისტიკაში ვხედავთ, რომ მათი საშუალო და მედიანა ერთმანეთთან ასე თუ ისე ახლოსაა. რადგანაც ყველა ცვლადი, სადაც გამოტოვებული მნიშვნელობა გვაქვს, შეგვიძლია მივიჩნიოთ უწყვეტ მონაცემად, გამოვიყენოთ მედიანა გამოტოვებული მნიშვნელობების შესავსებად:

In [ ]:
df_copy = df.copy()

for column in df_copy.columns:
    if df_copy[column].isna().sum() > 0:
        df_copy.fillna({column: df_copy[column].median()}, inplace=True)

In [ ]:
df_copy.isna().sum()

აუცილებელია მონაცემების სკალირება/სტანდარტიზაცია, სანამ გამოვიყენებთ მანძილზე დაფუძნებულ ტექნიკებს, როგორიცაა PCA ან UMAP:

In [ ]:
X_copy_scaled = StandardScaler().fit_transform(df_copy.drop(columns=["Outcome"]))

In [ ]:
pca = PCA(n_components=2, random_state=21)
pca_reduced_X = pca.fit_transform(X_copy_scaled)

plt.figure(figsize=(8, 8))
sns.scatterplot(x=pca_reduced_X[:, 0], y=pca_reduced_X[:, 1], hue=df_copy["Outcome"])
plt.show()

In [ ]:
umap_obj = umap.UMAP(n_components=2, n_neighbors=5, n_jobs=1, random_state=21)
umap_reduced_X = umap_obj.fit_transform(X_copy_scaled)

plt.figure(figsize=(8, 8))
sns.scatterplot(x=umap_reduced_X[:, 0], y=umap_reduced_X[:, 1], hue=df_copy["Outcome"])
plt.show()

In [ ]:
del df_copy

მონაცემების 80% გამოვიყენოთ HPO-სთვის და სატრენინგოდ, ხოლო დარჩენილი 20% სატესტოდ:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=["Outcome"]),
    df["Outcome"],
    test_size=0.2,
    random_state=21,
    stratify=df["Outcome"],
)

In [ ]:
print("სატრენინგო მონაცემების ზომა:", X_train.shape, y_train.shape)
print("სატესტო მონაცემების ზომა:", X_test.shape, y_test.shape)

## Logistic Regression vs. Decision Tree vs. Random Forest vs. XGBoost vs. LightGBM

დავალებაში მოცემული ალგორითმებიდან:

* Linear Regression
* Ridge
* PCA
* Logistic Regression
* Decision Tree
* Random Forest
* XGBoost
* LightGBM
* KNN
* K-Means
* SVM

შევადაროთ 5 ალგორითმი:

* Logistic Regression
* Decision Tree
* Random Forest
* XGBoost
* LightGBM

რა თქმა უნდა, ჰიპერპარამეტრების ოპტიმიზაციაც საჭიროა, თუმცა რესურსების შეზღუდვის გამო, ოდნავ მცირე ჰიპერპარამეტრების სივრცე გვექნება. შეფასების მეტრიკებად გამოვიყენებთ აკურატულობას, სიზუსტეს, გახსენებას, F1 ქულას და AUROC-ს. საუკეთესო მოდელსა და მის ჰიპერპარამეტრებს ამოვარჩევთ AUROC-ის საშუალებით, რადგანაც **ზედმეტად არადაბალანსირებული სამიზნე ცვლადი არ გვაქვს**.

გავამზადოთ საჭირო ფუნქციები, რომელთაც HPO-სთვის გამოვიყენებთ:

In [ ]:
def get_all_param_combinations(params_dict):
    keys = params_dict.keys()
    values = params_dict.values()

    # ყველა კომბინაციის დაგენერირება
    combinations = [dict(zip(keys, combination)) for combination in product(*values)]

    return combinations


def run_cv(alg, params, X, y, random_state=21):
    accuracies = {}
    precisions = {}
    recalls = {}
    f1s = {}
    aurocs = {}

    skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_state)

    for ind, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        X_train, X_valid = X.iloc[train_idx].copy(), X.iloc[valid_idx].copy()
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        # შევავსოთ გამოტოვებული მონაცემები სატრენინგოს მედიანათი
        for column in X_train.columns:
            if X_train[column].isna().sum() > 0 or X_valid[column].isna().sum():
                train_median = X_train[column].median()
                X_train.fillna({column: train_median}, inplace=True)
                X_valid.fillna({column: train_median}, inplace=True)

        # სტანდარტიზაცია სატრენინგო მონაცემების სტატისტიკით
        scaler = StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_valid = scaler.transform(X_valid)

        # მოდელის ობიექტის შექმნა
        if alg == "logistic_regression":
            clf = LogisticRegression(random_state=random_state, **params)
        elif alg == "decision_tree":
            clf = DecisionTreeClassifier(random_state=random_state, **params)
        elif alg == "random_forest":
            clf = RandomForestClassifier(random_state=random_state, **params)
        elif alg == "xgboost":
            clf = xgb.XGBClassifier(random_state=random_state, **params)
        elif alg == "light_gbm":
            clf = lgb.LGBMClassifier(random_state=random_state, **params)
        else:
            raise ValueError("Invalid algorithm")

        clf.fit(X_train, y_train)

        # შეფასების მეტრიკების გამოთვლა სატრენინგო და სავალიდაციო მონაცემებზე
        train_preds = clf.predict(X_train)
        valid_preds = clf.predict(X_valid)

        accuracies[f"accuracy_{ind}"] = [
            accuracy_score(y_train, train_preds),
            accuracy_score(y_valid, valid_preds),
        ]
        precisions[f"precision_{ind}"] = [
            precision_score(y_train, train_preds),
            precision_score(y_valid, valid_preds),
        ]
        recalls[f"recall_{ind}"] = [
            recall_score(y_train, train_preds),
            recall_score(y_valid, valid_preds),
        ]
        f1s[f"f1_{ind}"] = [
            f1_score(y_train, train_preds),
            f1_score(y_valid, valid_preds),
        ]
        aurocs[f"auroc_{ind}"] = [
            roc_auc_score(y_train, clf.predict_proba(X_train)[:, 1]),
            roc_auc_score(y_valid, clf.predict_proba(X_valid)[:, 1]),
        ]

    metrics_df = pd.DataFrame(
        dict(
            {"algorithm": [alg, alg]},
            **{
                "params": [
                    params | {"random_state": random_state},
                    params | {"random_state": random_state},
                ]
            },
            **{"set": ["Training", "Validation"]},
            **accuracies,
            **precisions,
            **recalls,
            **f1s,
            **aurocs,
        )
    )

    return metrics_df

ჰიპერპარამეტრების სივრცე თითოეული ალგორითმისთვის:

In [ ]:
algorithms_and_params = {
    "logistic_regression": {
        "penalty": ["l2"],
        "C": [0.5, 1, 5, 10],
    },
    "decision_tree": {
        "max_features": ["log2", "sqrt", None],
        "min_samples_leaf": [2, 4],
        "max_depth": np.arange(3, 11),
    },
    "random_forest": {
        "n_estimators": np.arange(10, 60, 10),
        "max_features": ["log2", "sqrt", None],
        "min_samples_leaf": [2, 4],
        "max_depth": np.arange(3, 11),
    },
    "xgboost": {
        "objective": ["binary:logistic"],
        "n_estimators": np.arange(10, 60, 10),
        "max_leaves": np.arange(10, 30, 5),
        "max_depth": np.arange(3, 11),
    },
    "light_gbm": {
        "boosting_type": ["gbdt"],
        "objective": ["binary"],
        "force_col_wise": [True],
        "deterministic": [True],
        "verbose": [-1],
        "n_estimators": np.arange(10, 60, 10),
        "num_leaves": np.arange(10, 30, 5),
        "max_depth": np.arange(3, 11),
    },
}

თითოეული კომბინაციით ალგორითმის ტრენინგი და შედეგების შენახვა:

In [ ]:
start_time = datetime.datetime.now()

metrics_dfs = []

for alg, params_dict in algorithms_and_params.items():
    params_list = get_all_param_combinations(params_dict)

    for params in params_list:
        metrics_df = run_cv(alg, params, X_train, y_train)
        metrics_dfs.append(metrics_df)

    print(f"{alg}: HPO is done.")

finish_time = datetime.datetime.now()
print(f"HPO is done in {finish_time - start_time}")

შედეგების გაერთიანება ერთ ცხრილად:

In [ ]:
master_metrics_df = pd.concat(metrics_dfs, ignore_index=True)

In [ ]:
len(master_metrics_df)

ჯამში ყველა მოდელისთვის გვქონდა ჰიპერპარამეტრების 612 კომბინაცია. რადგანაც როგორც სატრენინგო, ასევე სავალიდაციო ნაწილების შედეგებიც შევინახეთ, ამიტომ ცხრილში გვაქვს 1224 ჩანაწერი.

თითოეული შეფასების მეტრიკისთვის სატრენინგო და სავალიდაციო ნაწილების საშუალო არითმეტიკულისა და სტანდარტული გადახრის გამოთვლა:

In [ ]:
master_metrics_df["accuracy_avg"] = master_metrics_df[
    ["accuracy_0", "accuracy_1", "accuracy_2"]
].mean(axis="columns")
master_metrics_df["accuracy_std"] = master_metrics_df[
    ["accuracy_0", "accuracy_1", "accuracy_2"]
].std(axis="columns")

master_metrics_df["precision_avg"] = master_metrics_df[
    ["precision_0", "precision_1", "precision_2"]
].mean(axis="columns")
master_metrics_df["precision_std"] = master_metrics_df[
    ["precision_0", "precision_1", "precision_2"]
].std(axis="columns")

master_metrics_df["recall_avg"] = master_metrics_df[
    ["recall_0", "recall_1", "recall_2"]
].mean(axis="columns")
master_metrics_df["recall_std"] = master_metrics_df[
    ["recall_0", "recall_1", "recall_2"]
].std(axis="columns")

master_metrics_df["f1_avg"] = master_metrics_df[["f1_0", "f1_1", "f1_2"]].mean(
    axis="columns"
)
master_metrics_df["f1_std"] = master_metrics_df[["f1_0", "f1_1", "f1_2"]].std(
    axis="columns"
)

master_metrics_df["auroc_avg"] = master_metrics_df[
    ["auroc_0", "auroc_1", "auroc_2"]
].mean(axis="columns")
master_metrics_df["auroc_std"] = master_metrics_df[
    ["auroc_0", "auroc_1", "auroc_2"]
].std(axis="columns")

საშუალო AUROC-ზე დაყრდნობით სავალიდაციო მონაცემებში საუკეთესო ალგორითმისა და მისი ჰიპერპარამეტრების ამორჩევა:

In [ ]:
best_params = master_metrics_df.loc[
    master_metrics_df[master_metrics_df["set"] == "Validation"]["auroc_avg"].argmax()
]["params"]

In [ ]:
master_metrics_df[master_metrics_df["params"] == best_params]

In [ ]:
best_params

როგორც შედეგებიდან ჩანს, საუკეთესო მოდელია ლოჯისტიკური რეგრესია.

მაინც დავაკვირდეთ კიდევ სხვა რომელიმე მოდელს ხომ არ აქვს ვალიდაციის მონაცემებზე 0.83-ზე მეტი AUROC:

In [ ]:
master_metrics_df[
    (master_metrics_df["set"] == "Validation") & (master_metrics_df["auroc_avg"] > 0.83)
]

## საბოლოო მოდელის ტრენინგი და შეფასება

რა თქმა უნდა, მონაცემები მსგავსად უნდა დამუშავდეს, როგორც ჯვარედინი ვალიდაციისას მოხდა:

In [ ]:
for column in X_train.columns:
    if X_train[column].isna().sum() > 0 or X_test[column].isna().sum() > 0:
        train_median = X_train[column].median()
        X_train.fillna({column: train_median}, inplace=True)
        X_test.fillna({column: train_median}, inplace=True)

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

დავატრენინგოთ ლოჯისტიკური რეგრესია საუკეთესო ჰიპერპარამეტრებით:

In [ ]:
log_reg = LogisticRegression(**best_params)
log_reg.fit(X_train, y_train)

ვნახოთ მისი შეფასების მეტრიკები როგორც სატრენინგო, ასევე სატესტო მონაცემებზე:

In [ ]:
train_preds = log_reg.predict(X_train)
test_preds = log_reg.predict(X_test)

metrics_df = pd.DataFrame(
    {
        "accuracy": [
            accuracy_score(y_train, train_preds),
            accuracy_score(y_test, test_preds),
        ],
        "precision": [
            precision_score(y_train, train_preds),
            precision_score(y_test, test_preds),
        ],
        "recall": [
            recall_score(y_train, train_preds),
            recall_score(y_test, test_preds),
        ],
        "f1": [f1_score(y_train, train_preds), f1_score(y_test, test_preds)],
        "auroc": [
            roc_auc_score(y_train, log_reg.predict_proba(X_train)[:, 1]),
            roc_auc_score(y_test, log_reg.predict_proba(X_test)[:, 1]),
        ],
    },
    index=["Training", "Test"],
)
metrics_df

დავაკვირდეთ დაბნეულობის მატრიცას სატრენინგო და სატესტო მონაცემებზე:

In [ ]:
fig, (ax_1, ax_2) = plt.subplots(1, 2, figsize=(14, 7))

train_cm = confusion_matrix(y_train, train_preds)
sns.heatmap(train_cm, annot=True, cmap="Blues", fmt="g", square=True, ax=ax_1)
ax_1.tick_params(axis="y", labelrotation=0)
ax_1.set_title("Training")
ax_1.set_ylabel("Actual Label")
ax_1.set_xlabel("Predicted Label")


test_cm = confusion_matrix(y_test, test_preds)
sns.heatmap(test_cm, annot=True, cmap="Blues", fmt="g", square=True, ax=ax_2)
ax_2.tick_params(axis="y", labelrotation=0)
ax_2.set_title("Test")
ax_2.set_ylabel("Actual Label")
ax_2.set_xlabel("Predicted Label")

plt.show()

დავაკვირდეთ ROC მრუდს სატრენინგო და სატესტო მონაცემებზე:

In [ ]:
plt.figure(figsize=(10, 6))

y_train_pred_proba = log_reg.predict_proba(X_train)[:, 1]
train_fpr, train_tpr, _ = roc_curve(y_train, y_train_pred_proba)
train_auc = roc_auc_score(y_train, y_train_pred_proba)
plt.plot(train_fpr, train_tpr, label=f"Training (AUC={train_auc:.2f})")

y_test_pred_proba = log_reg.predict_proba(X_test)[:, 1]
test_fpr, test_tpr, _ = roc_curve(y_test, y_test_pred_proba)
test_auc = roc_auc_score(y_test, y_test_pred_proba)
plt.plot(test_fpr, test_tpr, label=f"Test (AUC={test_auc:.2f})")

plt.plot([0, 1], [0, 1], linestyle="--", color="grey", label="Random (AUC=0.5)")

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.legend(loc="lower right")
plt.show()